# 7 v2

## Import

In [1]:
# import logging
from pathlib import Path
import torch
import os

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, PromptTuningConfig

## Initialization

In [2]:
def find_project_root() -> Path:

    start_path = Path.cwd()
    for parent in start_path.parents:
        if (parent / ".git").exists() or (parent / "pyproject.toml").exists() or (parent / "setup.py").exists():
            return parent
    return start_path  # Fallback: if no marker is found, return the original path


# Get the project root automatically
PROJECT_ROOT = find_project_root()
print(Path.cwd())

d:\Projects\finam_hotel_reviews\notebooks


In [3]:
# logging.basicConfig(
#     level=logging.INFO,
#     format="%(asctime)s - %(name)s -%(levelname)s - %(funcName)s -  %(message)s",
#     handlers=[logging.StreamHandler(), logging.FileHandler(PROJECT_ROOT / "logs" / "logfile.log")],
# )

device = torch.device("cuda") if torch.cuda.is_available else torch.device("cpu")

## Load model

In [3]:
model_name = "lightblue/DeepSeek-R1-Distill-Qwen-7B-Multilingual"

# Настройка 4-битной квантования
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16  # Use float16 for faster computation
)

# Загрузка токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map="auto")

# Настройка Prompt Tuning
prompt_tuning_config = PromptTuningConfig(
    task_type="CAUSAL_LM", num_virtual_tokens=20  # Тип задачи  # Количество виртуальных токенов для оптимизации
)

# Применение Prompt Tuning
model = get_peft_model(model, prompt_tuning_config)
# logging.info(model.print_trainable_parameters())
print(model.print_trainable_parameters())

# logging.info(" DeepSeek LLM загружен с Prompt Tuning и 4-битным квантованием!")
print(" DeepSeek LLM загружен с Prompt Tuning и 4-битным квантованием!")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 71,680 || all params: 7,615,688,192 || trainable%: 0.0009
None
 DeepSeek LLM загружен с Prompt Tuning и 4-битным квантованием!


## Load and Preprocess Dataset

In [5]:
import pandas as pd
import json

system_prompt = "Ты - аналитие по анализу отзывов. Разметь данный отзыв об отеле, выделяя все возможные аспекты и их тональность. Ответ в формате json.:"
df_initial = pd.read_csv(PROJECT_ROOT / "data" / "interim" / "200_labeled_gpt_4.csv")
df_initial.head(2)

ls_labels_raw = df_initial["labels"].to_list()

ls_aspect = []
ls_sentiment = []
ls_text = df_initial["text"].to_list()
ls_labels = []
ls_big_text = []

for label in ls_labels_raw:
    label_json = json.loads(label.replace("'", '"'))
    ls_labels.append(label_json)

for text, label in zip(ls_text, ls_labels_raw):
    fin_text = system_prompt + f"``` {text} ```  ответ: ``` {label} ```"
    ls_big_text.append(fin_text)

In [11]:
from datasets import Dataset, DatasetDict
import random

data = {"text": ls_text, "labels_raw": ls_labels_raw, "labels_json": ls_labels, "text_prompt_label": ls_big_text}

# Создаем Dataset
dataset = Dataset.from_dict(data)

# Разделяем на train/test
dataset = dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)

# Создаем DatasetDict
# dataset_dict = DatasetDict(
#     {"train": dataset["train"], "test": dataset["test"], "text_prompt_label": dataset["text_prompt_label"]}
# )

# # Проверяем структуру
# print(dataset_dict)

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels_raw', 'labels_json', 'text_prompt_label'],
        num_rows: 160
    })
    test: Dataset({
        features: ['text', 'labels_raw', 'labels_json', 'text_prompt_label'],
        num_rows: 40
    })
})

## Tokenize dataset

In [13]:
def tokenize_function(examples):
    inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    inputs["labels"] = inputs["input_ids"].copy()  # Use input_ids as labels for causal LM
    return inputs


tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Verify tokenized sample
print("Tokenized Sample with Labels:")
print(tokenized_datasets["train"][0])

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Tokenized Sample with Labels:
{'text': 'Невероятный отель с лучшими номерами. Чего стоят только эти матрасы, на которые ложишься и не можешь встать. В самом центре города. Шоппинг, достопримечательности - все рядом. Завтраки наивкуснейшие! Очень советую любителям шикарно отдохнуть!', 'labels_raw': "[{'aspect': 'общее впечатление', 'sentiment': 'positive'}, {'aspect': 'номер', 'sentiment': 'positive'}, {'aspect': 'матрасы', 'sentiment': 'positive'}, {'aspect': 'расположение', 'sentiment': 'positive'}, {'aspect': 'достопримечательности', 'sentiment': 'positive'}, {'aspect': 'завтрак', 'sentiment': 'positive'}]", 'labels_json': [{'aspect': 'общее впечатление', 'sentiment': 'positive'}, {'aspect': 'номер', 'sentiment': 'positive'}, {'aspect': 'матрасы', 'sentiment': 'positive'}, {'aspect': 'расположение', 'sentiment': 'positive'}, {'aspect': 'достопримечательности', 'sentiment': 'positive'}, {'aspect': 'завтрак', 'sentiment': 'positive'}], 'text_prompt_label': "Ты - аналитие по анализу отз

## Set training parameters

In [21]:
import os

os.environ["WANDB_DISABLED"] = "true"
os.environ["CLEARML_DISABLED"] = "true"
os.environ["DVCLIVE_DISABLED"] = "true"


from transformers import TrainingArguments

# Training arguments
training_args = TrainingArguments(
    output_dir=PROJECT_ROOT / "results",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir=PROJECT_ROOT / "logs",
    fp16=True,
    report_to=[],  # Отключает интеграции (DVC, ClearML, WandB и т.д.)
)

# logging.info("WandB Disabled!")
print("WandB Disabled!")

WandB Disabled!


C:\Users\baksa\AppData\Roaming\Python\Python311\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Get sample Data

In [22]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10))

## Initialize trainer and train

In [23]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset.remove_columns(["text"]),  # Remove raw text column
    eval_dataset=small_test_dataset.remove_columns(["text"]),
)

print("Trainer Initialized!")

Trainer Initialized!


In [24]:
torch.cuda.empty_cache()
print("Cleared CUDA Cache")

Cleared CUDA Cache


## Fine-Tune DeepSeek LLM

In [25]:
print("🚀 Starting Fine-Tuning...")
trainer.train()

🚀 Starting Fine-Tuning...


Epoch,Training Loss,Validation Loss
1,No log,7.551310
2,No log,7.323419
3,No log,7.122530
4,No log,6.959853


TrainOutput(global_step=60, training_loss=6.884547424316406, metrics={'train_runtime': 96.5585, 'train_samples_per_second': 5.178, 'train_steps_per_second': 0.621, 'total_flos': 1.0078517081407488e+16, 'train_loss': 6.884547424316406, 'epoch': 4.64})

## Run predictions

In [26]:
def generate_prediction(review_text):
    inputs = tokenizer(review_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


# Example reviews
reviews = [ls_text[0], ls_text[1], ls_text[2]]
reviews = [ls_text[0] + system_prompt, ls_text[1] + system_prompt, ls_text[2] + system_prompt]

# Run predictions
for review in reviews:
    print(f"Review: {review}")
    print(f"Predicted Sentiment: {generate_prediction(review)}")
    print("-" * 80)

Review: Это был самый маленький по площади номер из всех отелей, где я останавливалась. Тем не менее, очень уютненький и комфортабельный мини отельчик. Мебель новая, все функционирует, удобно расположен. Учитывая дороговизну отелей Парижа, для эконом варианта вполне достойно.Ты - аналитие по анализу отзывов. Разметь данный отзыв об отеле, выделяя все возможные аспекты и их тональность. Ответ в формате json.:


C:\Users\baksa\AppData\Roaming\Python\Python311\site-packages\peft\peft_model.py:1889: UserWarning: Position ids are not supported for parameter efficient tuning. Ignoring position ids.
  warnings.warn("Position ids are not supported for parameter efficient tuning. Ignoring position ids.")


Predicted Sentiment: Это был самый маленький по площади номер из всех отелей, где я останавливалась. Тем не менее, очень уютненький и комфортабельный мини отельчик. Мебель новая, все функционирует, удобно расположен. Учитывая дороговизну отелей Парижа, для эконом варианта вполне достойно.Ты - аналитие по анализу отзывов. Разметь данный отзыв об отеле, выделяя все возможные аспекты и их тональность. Ответ в формате json.: {"name": "Отель number", "address": "number", "rating": "number", "description": "text", "category": "text", "toxic": "toxic", "toxic_weight": "toxic_weight", "toxic_type": "toxic_type"}: json: {"name": "Отель number", "address": "number", "rating": "number", "description": "text", "category": "text", "toxic": "toxic", "toxic_weight": "toxic_weight", "toxic_type": "toxic_type"}: json: {"name": "Отель number", "address": "number", "rating": "number", "description": "text", "category": "text", "toxic": "toxic", "toxic_weight": "toxic_weight", "toxic_type": "toxic_type"}: